# Scikit Learn Re-usable Component (local train)

In [ ]:
# Set your output and project. !!!Must Do before you can proceed!!!
YOUR_NAME = 'Erwin' #change to your name
EXPERIMENT_NAME = 'Scikit Learn V1.14 %s' % (YOUR_NAME) # change to your experiment name
PROJECT_NAME =  'kfp-primer-workshop' # change this to your project name   

### Create KFP Experiment

In [ ]:
import kfp
from kfp import gcp
import os
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

### Load component from file

Here we are loading the Scikit Learn component. 

In [ ]:
dummy_op = kfp.components.load_component_from_file('components/scikit_component/component.yaml')

In [ ]:
### Author you pipeline using the dummy_op we just created. 

In [ ]:
from kfp import dsl

# Define a pipeline and create a task from a component:
@kfp.dsl.pipeline(name='KFP test erwinh', description='')
def my_pipeline(gcs_path_data = dsl.PipelineParam(name='pathdata', value='gs://erwinh-public-data/scikit/data/scikit_marketing-data.csv'),
               data_storage = dsl.PipelineParam(name='storage', value='BQ'),
               gcs_path_model = dsl.PipelineParam(name='pathoutput', value='gs://kfp-scikit/model/output/'),
               bigquery_table = dsl.PipelineParam(name='bqtable', value='kfp-primer-workshop.marketing_data.raw')
               ):
    dummy1_task = dummy_op(
        # Input name "Input 1 URI" is converted to pythonic parameter name "input_1_uri"
        pathdata = gcs_path_data,
        storage = data_storage,
        pathoutput =gcs_path_model,
        bqtable = bigquery_table,
        ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 


### Compile the pipeline created (create zip file). 

In [ ]:
pipeline_func = my_pipeline  
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

### Submit the compile pipeline to run

In [ ]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)